# Eksploracja Krajowych planów na rzecz energii i klimatu

## Co wyniknęło z eksploracji? 
- Głównym krokiem przed eksploracją było przetworzenie danych w odpowiedni sposób. Uwzględniona została przy tym zakładana struktura dokumentów. Okazało się, że rzeczywiście występują istotne różncie pomiędzy poszczególnymi sekcjami i wymiarami, zatem analiza w podziale na składowe dokumentu ma sens.
- Okazało się również, że w danych widać różnice pomiędzy dokumentami poszczególnych państw, co także stanowi dobry znak dla dalszej pracy zakładającej dokładniejsze porównania między poszczególnymi członkami UE. 
- Pojawiły się kolejne pytania badawcze, np. dotyczące różnic w traktowaniu o transporcie w procesie dekarbonizacji. 
- Zidentyfikowano słowa, które należy rozważyć w kontekście uwzględnienia jako stop-słowa.
- Wskazano dalsze kroki: próba poprawy odczytu tekstów z PDF (bez tabel, wykresów, numeracji stron)

## Importy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python -m spacy download en_core_web_md
# trzeba uruchomić ponownie środowisko wykonawcze po pobraniu

In [ ]:
# !pip install PyPDF2
# from PyPDF2 import PdfFileReader

In [ ]:
! pip install swifter
! pip install matplotlib==3.4.0
! pip install textacy
! pip install thinc

In [ ]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
en = spacy.load("en_core_web_md")

In [ ]:
import os
import swifter
import pickle
from collections import Counter
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

In [ ]:
DIR = '/content/drive/MyDrive/NLP-klimat/'

## Analiza

### Wczytanie danych 
Teksty zostały odczytane z PDF-ów na podstawie wcześniejszego otagowania  poszczególnych dokumentów. 

In [ ]:
NECP_annotations = pd.read_csv(DIR+'NECP.txt')

In [ ]:
NECP_annotations = NECP_annotations.replace({"None": None})

Tagowanie odbywało się ręcznie poprzez przegląd dokumentów i oznaczenie odpowiednich sekcji (więcej o sekcjach później, w rozdziale o analizach).

In [ ]:
NECP_annotations

In [ ]:
def read_NECPs(annotations_df, result_texts_list):
  for i, row in annotations_df.iterrows():
    if row.start_page is None:
      result_texts_list.append(None)
      continue
    start_page = int(row.start_page)
    end_page = int(row.end_page) 
    print(f"Row {i} - reading {int(end_page-start_page+1)} pages from file: {row.file_name}; subsection: {row.subsection}, dimension: {row.energy_union_dimension}")
    if i > 0:  
      if row.file_name != current_file_name:
        current_file_name = row.file_name
    else:
      current_file_name = row.file_name
    file = open(DIR+'Final NECPs received/'+current_file_name, 'rb')
    fileReader = PdfFileReader(file)
    text = ""
    count = start_page-1
    while count < end_page:
      pageObj = fileReader.getPage(count)
      count += 1
      text += pageObj.extractText().replace("\n", "")
    if row.start_text is not None:
      text = row.start_text+text.split(row.start_text, 1)[1]
    if row.end_text is not None:
      text = text.rsplit(row.end_text, 1)[0]+row.end_text
    result_texts_list.append(text)
  return result_texts_list

In [ ]:
all_texts = []

In [ ]:
all_texts = read_NECPs(NECP_annotations, all_texts)

In [ ]:
NECP_annotations["text"] = all_texts

In [ ]:
NECP_annotations.to_csv("necp_processed.csv")

### Wstęp do analizy NECP

#### Intro: czym jest NECP?

**NECP** - National Energy and Climate Plan (Krajowy plan na rzecz energii i klimatu)

Aby zrealizować ustanowione przez Unię Europejską cele w zakresie energii i klimatu na 2030 rok, państwa członkowskie zostały zobowiązane do ustanowienia 10-letniego planu na rzecz energii i klimatu na okres od 2021 do 2030 roku (NECP).

**Struktura NECP**

![](https://i.ibb.co/nD280yN/necp-structure.png)


Zatem dla każdego z 27 państw członkowskich otrzymujemy sekcje:
- Overview and Process for Establishing the Plan - Zarys ogólny i proces opracowywania planu
- National Objectives and Targets - Krajowe założenia i cele
- Policies and Measures - Polityki i działania
- Current Situation and Reference Projections - Aktualna sytuacja i prognozy z obecną polityką klimatyczną
- Impact Assessment of Planned Policies and Measures - Ocena wpływu planowanych działań na rzecz klimatu

Według wzorcowej struktury sekcje 2-5 powinny byc podzielone na 5 wymiarów:
- Decarbonisation - Obniżenie emisyjności
- Energy efficiency - Efektywność energetyczna
- Energy security - Bezpieczeństwo energetyczne
- Internal market - Wewnętrzny rynek energii
- R&I and Competitiveness - Badania naukowe, innowacje i konkurencyjność

W rzeczywistości w większości planów w sekcji oceny wpływu planowanych działań na rzecz klimatu nie ma podziału na 5 wymiarów

In [ ]:
necp_processed = pd.read_csv(DIR+'necp_processed.csv', index_col = 0)

Kolumny zaimportowanej ramki danych.

In [ ]:
necp_processed.columns

Kolumny 'start_page', 'end_page', 'start_text', 'end_text' służyły jedynie do poprawnego wczytania dokumentów z PDF-ów, zatem możemy się ich pozbyć.

In [ ]:
necp_processed.drop(['start_page', 'end_page', 'start_text', 'end_text'], axis = 1, inplace = True)

Przyjrzyjmy się brakom danych.

In [ ]:
necp_processed.info()

Nie każda z sekcji jest podzielona na wymiary, zatem braki danych w kolumnie 'energy_union_dimension' są uzasadnione. Przyjrzyjmy się rekordom, w których brakuje tesktu.  

In [ ]:
necp_processed.loc[necp_processed.isnull()["text"]]

In [ ]:
necp_processed.loc[(necp_processed["country"] == "Greece") & (necp_processed["subsection"] == "Current Situation and Reference Projections")]

Plan klimatyczny Grecji nie zawiera wydzielonej sekcji o aktualnej sytuacji i prognozach z obecną polityką klimatyczną. Łotwa nie ma wymiaru o obniżeniu emisyjności w sekcji Politylki i działań. Rekordy te nie przenoszą żadnej informacji. Można się ich pozbyć.

In [ ]:
necp_processed.drop(necp_processed[necp_processed.isnull()["text"]].index, axis = 0, inplace = True)

In [ ]:
len(necp_processed)

Zostało 453 części dokumentów.

##### Przetworzenie tekstów

In [ ]:
tqdm.pandas()
necp_docs = necp_processed['text'].swifter.apply(en)

In [ ]:
# eksport przetworzonych dokumentów
with open(DIR + 'necp_docs.pickle', 'wb') as f:
  pickle.dump(necp_docs, f)

##### Wczytanie

In [ ]:
# import przetworzonych dokumentów
with open(DIR + 'necp_docs.pickle', 'rb') as f:
    necp_docs = pickle.load(f)

### Analiza ilościowa

#### Długość dokumentów
liczona jako liczba słów w dokumencie

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(15,6))
necp_processed["length"] = necp_docs.str.len()

necp_processed["length"].hist(bins = 50)
plt.show()

Większość wydzielonych części dokumentów ma mniej niż 20 tysięcy słów. Przyjrzyjmy się wyjątkowo długim tekstom.

In [ ]:
necp_processed.loc[necp_processed["length"] > 30000][["country", "file_name", "subsection", "energy_union_dimension", "length"]]

Najdłuższe części (powyżej 30000 słów) to wybrane wymiary polityki i działań w planach Belgii i Litwy oraz sekcja oceny wpływu planowanych działań w Polsce.

Przyjrzyjmy się, czy można zaobserwować różnice w długościach poszczególnych sekcji/wymiarów. 

In [ ]:
plt.figure(figsize=(16, 8))
sns.boxplot(y='subsection', x='length', data=necp_processed)
plt.show()

Wyraźnie dłuższe są sekcje *Overview...* i *Impact...*, co ma jednak związek z tym, że dokumenty do nich należące nie są dzielone na zdefiniowane wymiary. Natomiast spośród sekcji dzielonych na wyniary, najdłuższe dokumenty dotyczą polityk i środków. 

In [ ]:
plt.figure(figsize=(16, 8))
sns.boxplot(y='energy_union_dimension', x='length', data=necp_processed)
plt.show()

Analizując poszczególne wymiary, widzimy, że najwięcej uwagi jest poświęcanej emisyjności - dekarbonizacji. Stosunkowo krótsze od pozostałych są rozdziały poświęcone bezpieczeństwu energetycznemu i researchowi. 

In [ ]:
plt.figure(figsize=(16, 8))
sns.boxplot(hue='energy_union_dimension', x='length', y='subsection', 
            data=necp_processed[~necp_processed.subsection.isin(
                ['Overview and Process for Establishing the Plan',
                'Impact Assessment of Planned Policies and Measures'])])
plt.show()

Obserwacje odnośnie różnic w długości potwierdzają się też kiedy popatrzymy na hierarchiczne pogrupowanie - najdluższe teksty są związane z Policies and Measures, przy czym to dekarbonizacja zajmuej najwięcej uwagi. 

#### Najczęstsze słowa

In [ ]:
def plot_counter(counter: Counter, orient: str = 'h', color: str='lightblue', figsize: tuple=(20,13)):
  plt.figure(figsize=figsize)
  keys = [k[0] for k in counter]
  vals = [int(k[1]) for k in counter]
  ax = sns.barplot(x=vals, y=keys, orient=orient, color=color)
  ax.bar_label(ax.containers[0])
  return ax

In [ ]:
necp_processed["necp_lemmas"] = necp_docs.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct if token.is_alpha])
necp_lemmas_counter = Counter(necp_processed["necp_lemmas"].sum()).most_common(30)

##### **Top 30 najczęstszych słów**
- we wszystkich dokumentach
- po lematyzacji

In [ ]:
plot_counter(necp_lemmas_counter)
plt.show()

Widzimy słowa, które ze względu na specyfikę tekstu będą się częściej powtarzać. Nie bierzmy ich pod uwagę w dalszej analizie - to pierwsi kandydaci na stop-słowa.

In [ ]:
extra_stop_words =  ['energy', 'Energy', 'measure', 'electricity', 'sector', 'EU', 
                     'system', 'market' ,'national', 'plan', 'project', 'use']
necp_processed["necp_lemmas"] = necp_docs.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct if token.is_alpha if not (token.lemma_ in extra_stop_words)])
necp_lemmas_counter2 = Counter(necp_processed["necp_lemmas"].sum())

In [ ]:
plot_counter(necp_lemmas_counter2.most_common(30))
plt.show()

Najczęściej występują takie słowa jak 'gas', 'emission', 'renewable'. Nie jest to zaskakujące. Bardzo prawdopodobne jest to, że  w krajowych planach na rzecz energii i klimatu będą występować słowa dotyczące przykładowo emisji gazów (lub gazu jako paliwo) i odnawialnych źródeł energii.

##### Top 30 najczęstszych noun-chunków 
- we wszystkich dokumentach


In [ ]:
chunks_list = []
for doc in necp_docs:
  for chunk in doc.noun_chunks:
    chunks_list.append(chunk.text)

chunk_unique, chuck_counts = np.unique(chunks_list, return_counts = True)
chunk_arr = np.c_[chunk_unique[chuck_counts.argsort()[::-1]], chuck_counts[chuck_counts.argsort()[::-1]]]

In [ ]:
plot_counter(chunk_arr[:30])
plt.show()

Wśród najczęstszych chunków, pojawiają się słowa, które uznaliśmy za występujące wyjątkowo często w NECP-ach. Dodatkowo pojawiają się takie frazy jak: 'the developement' - rozwój, 'energy efficiency' - wydajność energetyczna, czy 'RES' - skrót od Renewable Energy Sources.

Przeprowadźmy jeszcze Entity Recognition.

In [ ]:
ent_type_list = []
for doc in necp_docs:
  for ent in doc.ents:
      ent_type_list.append((ent.label_))

ent_type_unique, ent_type_counts = np.unique(ent_type_list, return_counts = True)
ent_type_counter = np.c_[ent_type_unique[ent_type_counts.argsort()[::-1]], ent_type_counts[ent_type_counts.argsort()[::-1]]]

In [ ]:
plot_counter(ent_type_counter)
plt.show()

W tekstach występuje najwięcej liczb, organizacji, dat i państw.

In [ ]:
ents_list = []
for doc in necp_docs:
  for ent in doc.ents:
    if ent.label_ not in ['CARDINAL', 'ORDINAL', 'DATE', 'GPE', 'MONEY', 'PERCENT', 'QUANTITY']:
      ents_list.append((ent.text))

ents_unique, ents_counts = np.unique(ents_list, return_counts = True)
ents_counter = np.c_[ents_unique[ents_counts.argsort()[::-1]], ents_counts[ents_counts.argsort()[::-1]]][:25]

In [ ]:
plot_counter(ents_counter)
plt.show()

Ponownie bez zaskoczeń. Najczęściej wspominana jest Unia Europejska, odnawialne źródła energii i krajowy plan na rzecz energii i klimatu. Rozszyfrujmy jeszcze inne występujące często skróty.

- GHG - greenhouse gas
- WEM - with existing measures (np. "with existing
measures" scenario)
- LULUCF - land use, land use change and forestry
- WAM -  with additional measures (np.  "with additional
measures" scenario)
- LNG - liquefied natural gas
- EC - często występująca część numeracji aktów prawnych Unii Europejskiej. Oznacza dyrektywę wiążącą państwa członkowskie do osiągnięcia pewnego rezultatu. Metody jakimi dane państwo cel osiągnie są dowolne.
- NEPN, INECP - Integrated National Energy and Climate Plan
- ECP - energy and climate policy

Dodatkowo warto zauważyć, że najczęściej wspominane zwroty dotyczące państw/regionów to Hungarian, Nordic, Polish, Baltic.

##### Najpopularniejsze słowa w podziale na sekcje

In [ ]:
sections = ["Overview and Process for Establishing the Plan", "National Objectives and Targets", "Policies and Measures",
            "Current Situation and Reference Projections", "Impact Assessment of Planned Policies and Measures"]
sections_counter = {}
for section in sections:
  necp_processed_temp = necp_processed.loc[necp_processed.subsection == section]
  cnt = Counter(necp_processed_temp["necp_lemmas"].sum())
  sections_counter[section] = cnt

In [ ]:
sections_counter_all = pd.DataFrame()
most_common_words = set()
for key, val in sections_counter.items():
  most_common_words.update({word for word, _ in val.most_common(10)})

In [ ]:
most_common_words = sorted(list(most_common_words))

In [ ]:
sections_len = necp_processed.groupby(['subsection']).length.sum()

In [ ]:
most_common_words_df = pd.DataFrame(columns=["word"]+list(sections_counter.keys()))
most_common_prom_words_df = pd.DataFrame(columns=["word"]+list(sections_counter.keys()))
for word in most_common_words:
  row = {'word':word}
  row2 = {'word':word}
  for section, counter in sections_counter.items():
    row[section] = counter[word]
    row2[section] = 1000*counter[word] / sections_len[section]
  most_common_words_df=most_common_words_df.append(row, ignore_index=True)
  most_common_prom_words_df=most_common_prom_words_df.append(row2, ignore_index=True)

**Liczba wystąpień popularnych słów w poszczególnych sekcjach** 
- suma zbiorów 10 najpopularniejszych słów w każdej sekcji
- pierwszy wykres liczby wystąpień
- drugi wykres liczby wystąpień w promilach (normowane przez łączną liczbę słów w dokumentach należących do danej sekcji)

In [ ]:
most_common_words_df=most_common_words_df.melt(id_vars="word", value_vars=sections)
sns.catplot(
    data=most_common_words_df, kind="bar",
    y="word", x="value", hue="variable",
    palette="dark", alpha=.6, height=12, orientation="horizontal"
)
plt.show()

In [ ]:
most_common_prom_words_df=most_common_prom_words_df.melt(id_vars="word", value_vars=sections)
sns.catplot(
    data=most_common_prom_words_df, kind="bar",
    y="word", x="value", hue="variable",
    palette="dark", alpha=.6, height=12, orientation="horizontal"
)
plt.show()

- można odczytać dość oczywiste rzeczy, jak to, że w sekcji Overview… stosunkowo bardziej popularne będą słowa jak climate czy plan, a w sekcji Impact Assessment… słowo scenario
- widzimy, że climate pojawia się stosunkowo rzadziej w sekcji Current Situation
- słowa consumption i gas są częściej wykorzystywane w sekcjach National Objectives i Current Situation
- słowa emission i fuel są częściej wykorzystywane w sekcjach Current Situation i Impact Assessment
- w sekcji Current Situation and Reference Projections częste jest słowo price
- support jest popularnym słowem w Policies and Measures
- renewable jest częste w National Objectives and Targets


##### Najpopularniejsze słowa w podziale na wymiary
- wykresy analogiczne jak powyżej

In [ ]:
dimensions_len = necp_processed.groupby(['energy_union_dimension']).length.sum()

In [ ]:
dimensions = [ 'Decarbonisation', 'Energy efficiency', 'Energy security', 'Internal market', 'R&I and Competitiveness']

dims_counter = {}
for dim in dimensions:
  necp_processed_temp = necp_processed.loc[necp_processed.energy_union_dimension == dim]
  cnt = Counter(necp_processed_temp["necp_lemmas"].sum())
  dims_counter[dim] = cnt

In [ ]:
dims_counter_all = pd.DataFrame()
most_common_words = set()
for key, val in dims_counter.items():
  most_common_words.update({word for word, _ in val.most_common(10)})
most_common_words = sorted(list(most_common_words))
most_common_words_df = pd.DataFrame(columns=["word"]+list(dims_counter.keys()))
most_common_prom_words_df = pd.DataFrame(columns=["word"]+list(dims_counter.keys()))
for word in most_common_words:
  row = {'word':word}
  row2 = {'word':word}
  for section, counter in dims_counter.items():
    row[section] = counter[word]
    row2[section] = 1000*counter[word] / dimensions_len[section]
  most_common_words_df=most_common_words_df.append(row, ignore_index=True)
  most_common_prom_words_df=most_common_prom_words_df.append(row2, ignore_index=True)

In [ ]:
most_common_words_df=most_common_words_df.melt(id_vars="word", value_vars=dimensions)
sns.catplot(
    data=most_common_words_df, kind="bar",
    y="word", x="value", hue="variable",
    palette="dark", alpha=.6, height=12, orientation="horizontal"
)
plt.show()

In [ ]:
most_common_prom_words_df=most_common_prom_words_df.melt(id_vars="word", value_vars=dimensions)
sns.catplot(
    data=most_common_prom_words_df, kind="bar",
    y="word", x="value", hue="variable",
    palette="dark", alpha=.6, height=12, orientation="horizontal"
)
plt.show()

W wymiarze obniżenia emisyjności dominują słowa: 'emission', 'renewable' i 'transport'. Możemy się domyślać, że państwa w dużej mierze planują obniżenie emisyjności w sektorze transportu. <br><br>
W wymiarze efektywności energetycznej pojawiają się takie słowa jak: 'building', 'public', 'renovation'. Może to sugerować potrzebę nowych inwestycji.<br><br>
W wymiarze bezpieczeństwa energetycznego ze specyficznych słów pojawiają się: 'natural' i 'capacity'. <br><br>
W wymiarze wewnętrznego rynku energii pojawiają się takie słowa jak: 'gas', 'transmission', 'network', 'supply'. W sekcji prawdopodbnie zawarte są informacje o transmisji energii. <br><br>
W wymiarze badań naukowych, innowacji i konkurencyjności znajdują się słowa w znacznym stopniu pasujące do tematu: 'research', 'technology', 'innovation', 'development', 'new'.

##### Analiza w podziale na kraje

In [ ]:
necp_processed.groupby(['country']).length.sum().sort_values(ascending=False)

Państwa, które stworzyły najdłuższe NECP-y to Belgia i Polska. Najkrótsze dokumenty pochodzą z Łotwy i Finlandii.

In [ ]:
countries = sorted(necp_processed["country"].unique())

for country in countries:
  necp_processed_temp = necp_processed.loc[necp_processed.country == country]
  cnt = Counter(necp_processed_temp["necp_lemmas"].sum()).most_common(7)
  print(country)
  for i in range(7):
    print(cnt[i])

W zdecydowanej większośći krajów jednym z najpopularniejszych słów jest właśnie nazwa kraju. Poza tym bardzo często występują takie frazy jak wcześniej ('gas', 'climate', 'renewable', 'policy' itp.). Ciężko wyciągnąć z tego sensowne wnioski.

In [ ]:
most_common_words_to_exlude = list(dict(necp_lemmas_counter2.most_common(30)).keys())
for country in countries:
  necp_processed_temp = necp_processed.loc[necp_processed.country == country]
  cnt = Counter(necp_processed_temp["necp_lemmas"].sum()).most_common()
  print(country)
  count = 0 
  i = 0
  while count < 7:
    if cnt[i][0] not in most_common_words_to_exlude:
      print(cnt[i])
      count+=1
    i+=1



Pomijając 30 najpopulaniejszych słów widać już więcej różnic między państwami, co jest dobrym znakiem przed modelowaniem tematów i próbą znalezienia istotnych różnic pomiędzy planami poszczególnych państw członkowskich. 

#### Analiza n-gramów
Więcej kontekstu w to, o czym jest mowa w analizowanych tekstach daje nam analiza digramów i trigramów. 

##### digramy

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(stop_words = en.Defaults.stop_words, ngram_range=(2, 2))
count_vector = cv.fit_transform(necp_processed['text'].values)
sum_ngram = count_vector.sum(axis=0)
ngram_freq = [(ngram, sum_ngram[0, idx]) 
              for ngram, idx in cv.vocabulary_.items()]
ngram_freq = sorted(ngram_freq, key = lambda x: x[1], reverse=True)

In [ ]:
plot_counter(ngram_freq[:20])
plt.show()

##### trigramy

In [ ]:
cv3 = CountVectorizer(stop_words = en.Defaults.stop_words, ngram_range=(3, 3))
count_vector3 = cv3.fit_transform(necp_processed['text'].values)
sum_ngram3 = count_vector3.sum(axis=0)
ngram_freq3 = [(ngram, sum_ngram3[0, idx]) 
              for ngram, idx in cv3.vocabulary_.items()]
ngram_freq3 = sorted(ngram_freq3, key = lambda x: x[1], reverse=True)

In [ ]:
plot_counter(ngram_freq3[:20])
plt.show()

Oprócz oczywistych wniosków o bliższym kontekście analizowanych tekstów, można zauważyć, że: 
- Na wykresach pojawiają się np. ważne horyzonty czasowe związane z Zielonym Ładem. 
- Potwierdzają się przypuszczenia o pojawianiu się słowa 'gas' w dwóch kontekstach. 

### Spoza kamienia milowego - przykład modelowania

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
necp_processed['lemmas'] = necp_docs.apply(lambda d: [t.lemma_ for t in d if not t.is_stop if t.is_alpha])
dictionary = Dictionary(necp_processed['lemmas'])
encoded_docs = necp_processed['lemmas'].apply(dictionary.doc2bow)

In [ ]:
lda = LdaMulticore(encoded_docs, num_topics=6)

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, encoded_docs, dictionary=dictionary)
vis